In [0]:
import contextlib
import io

@contextlib.contextmanager
def suppress_stdout_stderr():
    """Context manager to suppress stdout and stderr."""
    with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
        yield

def install_uv():
    with suppress_stdout_stderr():
        get_ipython().run_line_magic("pip", "install uv")

In [0]:
import os

def find_pyproject_toml(start_path='') -> str:
    """Search for pyproject.toml starting from start_path and moving up until found or root is reached."""

    if start_path:
        current_path = start_path
    else:
        current_path = "/Workspace" + \
            dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
 
    while True:
        possible_file = os.path.join(current_path, "pyproject.toml")
        if os.path.isfile(possible_file):
            return possible_file  # Found the file

        # Move up one directory
        parent_path = os.path.dirname(current_path)

        # Stop if we have reached the root directory
        if parent_path == current_path:
            raise FileNotFoundError("pyproject.toml not found in any parent directories.")

        current_path = parent_path
    
    return current_path


def find_project(start_path='') -> str:
    """Search for project_path from start_path and moving up until found or root is reached."""
    return os.path.dirname(find_pyproject_toml(start_path))

In [0]:
import re
import pandas as pd

def extract_toml_dependencies(pyproject_path: str) -> list[str]:
    import tomllib
    with open(pyproject_path, "rb") as f:
        data = tomllib.load(f)

    # Extract dependencies from standard format
    dependencies = data.get("project", {}).get("dependencies", [])

    return dependencies

def parse_toml_dependency(dep: str) -> tuple[str, str, str]:
    match = re.match(r"([a-zA-Z0-9_-]+)([<>=!~]+)(.+)", dep)
    if match:
        return match.groups()
    else:
        return (dep, "", "")

def parsed_toml_dependencies(pyproject_path: str) -> list[(str, str, str)]:
    """Parse a dependency string into a tuple of (package, operator, version)."""
    return [parse_toml_dependency(d) for d in extract_toml_dependencies(pyproject_path)]


def toml_dependencies_as_pdf(pyproject_path: str) -> pd.DataFrame:
    dep = parsed_toml_dependencies(pyproject_path)
    dat = pd.DataFrame(dep, columns=['Package', 'Dep_Operator', 'Dep_Version'])
    return dat

In [0]:
from importlib.metadata import distributions
import pandas as pd

def get_installed_packages() -> dict:
    """Get a dictionary of installed packages and their versions."""
    installed_packages = {dist.metadata["Name"].lower(): dist.version for dist in distributions()}
    return installed_packages

def installed_packages_as_pdf():
    tpl = [(k,v) for k,v in get_installed_packages().items()]
    dat = pd.DataFrame(tpl, columns=['Package', 'Installed_Version'])
    return dat

In [0]:
def is_serverless() -> bool:
    try:
        _ = dbutils.spark.conf.get('spark.databricks.clusterUsageTags.clusterId')
        return False
    except Exception as e:
        return True

In [0]:
import sys

def get_notebook_python_bin_path() -> str:
    env_path = next((p for p in sys.path if 'pythonEnv-' in p), None).split('/')[:-3]
    return ('/').join([*env_path,'bin','python'])

def get_cluster_python_bin_path() -> str:
    env_path = next((p for p in sys.path if 'cluster_libraries' in p), None).split('/')[:-3]
    return ('/').join([*env_path,'bin','python'])

def get_python_bin_path(cluster_install = False) -> str:
    if cluster_install and not is_serverless():
        return get_cluster_python_bin_path()
    else:
        return get_notebook_python_bin_path()